In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
from torch import nn

In [ ]:
device = "cuda" if  torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
start = 0
end =  10
step = 0.02
X = torch.tensor(torch.arange(start, end, step).unsqueeze(dim=1))
torch.manual_seed(43)
weight = torch.randn(1, requires_grad=False, dtype=torch.float)

torch.manual_seed(34)
bias = torch.randn(1, requires_grad=False, dtype=torch.float)

y = X * weight + bias
X= X.to(device)
X[:5], y[:5], X.shape, y.shape, weight, bias, X.device

<ipython-input-4-e000d6fbd9dc>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(torch.arange(start, end, step).unsqueeze(dim=1))


(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800]], device='cuda:0'),
 tensor([[1.4219],
         [1.4090],
         [1.3960],
         [1.3830],
         [1.3701]]),
 torch.Size([500, 1]),
 torch.Size([500, 1]),
 tensor([-0.6484]),
 tensor([1.4219]),
 device(type='cuda', index=0))

In [ ]:
seventy = int(0.7 * len(X))
X_train, X_test, y_train, y_test = X[:seventy], X[seventy:], y[:seventy], y[seventy:]
y_train = y_train.to(device)
y_test = y_test.to(device)
X_train.device,X_test.device, y_train.device,y_test.device, X_train.shape, X_test.shape, y_train.shape, y_test.shape

(device(type='cuda', index=0),
 device(type='cuda', index=0),
 device(type='cuda', index=0),
 device(type='cuda', index=0),
 torch.Size([350, 1]),
 torch.Size([150, 1]),
 torch.Size([350, 1]),
 torch.Size([150, 1]))

In [ ]:
class LinRegModel(nn.Module):
  def __init__(self) -> None:
    super().__init__()
    self.weights = nn.Parameter(torch.randn(1,
                                requires_grad=True,
                                dtype=torch.float)
                                )
    self.bias = nn.Parameter(torch.randn(1,
                                         requires_grad=True,
                                         dtype=torch.float)
                                )

  def forward(self, x:torch.tensor) -> torch.Tensor:
    return self.weights * x + self.bias   ## linear reg



## need model class to subclass if you want to load model params,
## since we oly saved params and not entire model
my_model = LinRegModel()

In [ ]:
from pathlib import Path
model_path = Path("/content/gdrive/MyDrive/Colab Notebooks/first_lin_regressor.pth")

my_model.load_state_dict(torch.load(model_path))                                ## takes pathlike object as input

<All keys matched successfully>

In [ ]:
my_model.state_dict()

OrderedDict([('weights', tensor([0.6993])), ('bias', tensor([0.3009]))])

In [ ]:
class LinRegModelV2(nn.Module):
  def __init__(self):
    super().__init__()

    self.linear_layer = nn.Linear(in_features=1,
                                  out_features=1)

  def forward(self, x:torch.Tensor) -> torch.Tensor:
    return self.linear_layer(x)


torch.manual_seed(43)
modelv2 = LinRegModelV2()
modelv2,  modelv2.state_dict()

(LinRegModelV2(
   (linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('linear_layer.weight', tensor([[-0.0921]])),
              ('linear_layer.bias', tensor([-0.6070]))]))

In [ ]:
next(modelv2.parameters()).device

device(type='cpu')

In [ ]:
modelv2.to(device)

LinRegModelV2(
  (linear_layer): Linear(in_features=1, out_features=1, bias=True)
)

In [ ]:
next(modelv2.parameters()).device, X_train.device

(device(type='cuda', index=0), device(type='cuda', index=0))

In [ ]:
loss_func = nn.L1Loss()

optimiser = torch.optim.SGD(params=modelv2.parameters(),
                            lr = 0.001)

In [ ]:
torch.manual_seed(43)

epochs = 100000

for epoch in range(epochs):
  modelv2.train()

  y_pred = modelv2(X_train)
  loss = loss_func(y_pred, y_train)
  optimiser.zero_grad()
  loss.backward()
  optimiser.step()
  modelv2.eval()

  with torch.inference_mode():
    y_pred_test = modelv2(X_test)
    test_loss = loss_func(y_pred_test, y_test)

  if epoch % 10000 == 0:
    print(epoch, test_loss)

0 tensor(2.6930, device='cuda:0')
10000 tensor(1.2281, device='cuda:0')
20000 tensor(0.9334, device='cuda:0')
30000 tensor(0.6387, device='cuda:0')
40000 tensor(0.3439, device='cuda:0')
50000 tensor(0.0488, device='cuda:0')
60000 tensor(0.0025, device='cuda:0')
70000 tensor(0.0025, device='cuda:0')
80000 tensor(0.0025, device='cuda:0')
90000 tensor(0.0025, device='cuda:0')


In [ ]:
list(modelv2.parameters())

[Parameter containing:
 tensor([[-0.6485]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([1.4218], device='cuda:0', requires_grad=True)]

In [ ]:
torch.save(obj=modelv2.state_dict(),
             f="gpu_trained_lin_reg.pth")

current timestamp - 8:32:00